# GP

What MSE does the GP achieve

In [ ]:
import torch

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel
from gpytorch.distributions import MultivariateNormal

from os.path import join

import sys
sys.path.append('../')
import utils

In [ ]:
assert torch.cuda.is_available(), 'CUDA is not available.'

## Load Data

In [ ]:
fname = join('/home/squirt/Documents/data/weather_data/', 'all_data.h5')

In [ ]:
split = 0.5
train_data, test_data = utils.get_data(fname, split)

In [ ]:
def combine_data(data_tuple:tuple[torch.tensor]) -> tuple[torch.tensor]:
    '''
    Add Landmass to x data. Return x,y tensors
    Input:
        - data_tuple (tuple[torch.tensor]): tuple of landmass,x,y tensors
    '''
    l,x,y = data_tuple

    # Combine
    l = l.unsqueeze(1)
    x = torch.cat((l, x), 1)
    x = x.contiguous()
    y = y.contiguous()
    return (x,y)

In [ ]:
train_data = combine_data(train_data)
test_data = combine_data(test_data)

In [ ]:
x,y = train_data
x = x.view(-1, 71*3*2*2)
y = y.view(-1, 70*2*2*2)

x = x.float().cuda()
y = y.float().cuda()

In [ ]:
x_test, y_test = test_data
x_test = x_test.view(-1, 71*3*2*2)
y_test = y_test.view(-1, 70*2*2*2)

x_test = x_test.float().cuda()
y_test = y_test.float().cuda()

## Define GP

Define Model using gptorch

In [ ]:
class MultitaskGaussianModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGaussianModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=train_y.shape[-1]      )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=train_y.shape[-1], rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)




In [ ]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=y.shape[1])
likelihood = likelihood.cuda()

model = MultitaskGaussianModel(x, y, likelihood=likelihood)
model = model.float().cuda()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
mll = mll.cuda()

## Train Model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Training loop
model.set_train_data(x, y)
likelihood.train()

for epoch in range(10):
    # Train Model
    model.train()
    optimizer.zero_grad()
        
    # This is a key part: call your model on a batch
    output = model(x)
    loss = -mll(output, y)
    loss.backward()
        
    optimizer.step()
    print('Here')

    #Test Model
    model.eval()
    loss = torch.nn.functional.mse_loss(model(x_test).mean, y_test)
    print(f'Epoch {epoch} - Loss: {loss}')

In [ ]:
y.shape
